# This notebook will serve for the merging of datasets for the TFW project

In [ ]:
# Import necassary libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

ROOT = os.environ.get('PWD')


# the outcome of the merging will be a dataset with the name, df_master_tfw7

# we will start with the inquiries dataset as the source of the instances of inquiries we will use for our model. to this file we will add data related to each property that have collected from the listings and room features files

In [ ]:

# read in inquiries dataset

df_inquiries = pd.read_csv('../data/master_inquiries_20210715.csv', low_memory=False)


In [ ]:

df_inquiries.head(3)


In [ ]:
df_inquiries.info()

In [ ]:
# df_inquiries.drop(['inquiry_price','length_stay'],axis=1,inplace=True)

In [ ]:
df_inquiries.nunique()

In [ ]:
del df_inquiries['Unnamed: 0']

# read in listings dataset. note that listings has the master list of properties, which will reduce the number of observations in inquiries and room features. we need to merge appropriately



In [ ]:
# read in listings dataset

df_listings = pd.read_csv('../data/master_listings_20210723.csv', low_memory=False)


In [ ]:
del df_listings['Unnamed: 0']

In [ ]:
df_listings.drop(['customer_id','state','subscription'],axis=1,inplace=True)

In [ ]:
df_listings.nunique()

In [ ]:
# do the first actual merge

df_master_tfw1 = pd.merge(df_inquiries, df_listings, left_on='listing_id', right_on='listing_id', how='right')

In [ ]:
# check to see what the outcome looks like in df_master_tfw 

df_master_tfw1.nunique()

In [ ]:
df_master_tfw1.info()

# merge all of room features dataset into master

In [ ]:
# load room features dataset
df_room_features = pd.read_csv('../data/room_features20210719.csv')

In [ ]:
df_room_features.head()

In [ ]:
df_master_tfw2 = pd.merge(df_room_features, df_master_tfw1, left_on='listing_id', right_on='listing_id', how='right')

In [ ]:
df_master_tfw2.info()

In [ ]:
df_master_tfw2.nunique()

In [ ]:
# del df_master_tfw2['time']

In [ ]:
# set global default to be able to see all columns

pd.set_option('display.max_rows', None)

In [ ]:
df_master_tfw2.isna().sum()

In [ ]:
x = list(df_master_tfw2.columns.values)

In [ ]:
y = x[1:145]

In [ ]:
y

In [ ]:
# filling in missing values for all the columns in the list 
for i in y:
    df_master_tfw2[i].fillna(0,inplace=True)

In [ ]:
df_master_tfw2.isna().sum()

In [ ]:
# imputing missing values for adult count, using median

df_master_tfw2['adult_count'] = df_master_tfw2['adult_count'].fillna(df_master_tfw2['adult_count'].median())

In [ ]:
# imputing missing values for children count, using median

df_master_tfw2['children_count'] = df_master_tfw2['children_count'].fillna(df_master_tfw2['children_count'].median())

In [ ]:
# imputing missing values for pets, using median

df_master_tfw2['pets_count'] = df_master_tfw2['pets_count'].fillna(df_master_tfw2['pets_count'].median())

In [ ]:
# delete the 655 rows that just shouldn't be there

df_master_tfw2.dropna(inplace=True)

In [ ]:
df_master_tfw2.isna().sum()

In [ ]:
df_master_tfw2.nunique()

In [ ]:
# for the time being, we will get rid of room features that are not common--with less than 1000 properties showing them, as they are resource intensive and are not likley to conbribute to the model

a_list = ['1_room_apartment', 'library', 'en_suite_bathroom', 'gallery', 'heating_room', 'lounge', 'roof-deck', 'veranda', 'winter_garden', 'VCR', 'alarm_system', 'baby_monitor', 'bathrobe', 'bidet', 'boat', 'boat_dock', 'bottle_warmer', 'changing_mat', 'computer', 'darts', 'exercise_room', 'fan', 'fax', 'folding_bed', 'game_console', 'guest_toilet', 'hot_tub', 'infrared_cabin', 'kingsize_bed', 'loft_bed', 'outdoor_shower', 'piano', 'pier', 'pool_table', 'relaxation_room', 'room_service', 'shared_swimming_pool', 'shoe_cabinet', 'steam_bath', 'swimming_pool', 'table_football', 'tanning_bed', 'terrace', 'wall_bed', 'water_bed']

for i in a_list:
    df_master_tfw2.drop([i],axis=1,inplace=True)

In [ ]:
# temp save for memory reasons

df_master_tfw2.to_csv('../data/master_2.csv')

In [ ]:
df_master_tfw2.nunique()

# replace inquiry price with relevant price data

In [ ]:
# read in relevant prices

df_new_price = pd.read_csv('../data/prices_for_master.csv')

In [ ]:
df_new_price.head(10)

In [ ]:
del df_new_price['Unnamed: 0']

In [ ]:
# we need to filter out price data that is not in our data range. create year column to do the filter

df_new_price['year'] = pd.to_datetime(df_new_price['date_from']).dt.year

In [ ]:
# drop prices that are outside of our data range

filter_price = df_new_price.query('year > 2018 & year < 2021')

In [ ]:
filter_price.head(100)

In [ ]:
filter_price.nunique()

# check shows only 2019 and 2020 are in the price data to be merged


In [ ]:
df_master_tfw2.head(2)

In [ ]:
df_master_tfw2.columns.values

In [ ]:
df_master_tfw4 = pd.merge(df_master_tfw2, filter_price, on='listing_id', how='outer')

In [ ]:
# save to csv

df_master_tfw4.to_csv('../data/master_4_temp.csv')



In [ ]:

# checking
 
df_master_tfw4.head()

In [ ]:
df_master_tfw4.shape

In [ ]:
# get rid of rows with missing price data

df_master_tfw4.dropna(inplace=True)

In [ ]:
df_master_tfw4.shape

In [ ]:
# extract price only on matches for arrival date in appropriate range

df_master_tfw5 = df_master_tfw4[(df_master_tfw4.arrival_date >= df_master_tfw4.date_from) & (df_master_tfw4.arrival_date < df_master_tfw4.date_to)]


In [ ]:
# checking

df_master_tfw5.shape

In [ ]:
# save intermediate data

df_master_tfw5.to_csv('../data/master_5.csv')

In [ ]:
# read master 5 from csv

df_master_tfw5 = pd.read_csv('../data/master_5.csv')

In [ ]:
# check for missing values

df_master_tfw5.isna().sum()

# data set has no missing values

In [ ]:
# checking merged set, columns look ok

df_master_tfw5.nunique()

In [ ]:
# get rid of year, because this is derived from a date other than that that we need

del df_master_tfw5['year']

In [ ]:
# generate year and month from arrival date for later uses

df_master_tfw5['year'] = pd.to_datetime(df_master_tfw5['arrival_date']).dt.year
df_master_tfw5['month'] = pd.to_datetime(df_master_tfw5['arrival_date']).dt.month

In [ ]:
# generate inquiry count for our data

df_master_inquiry_count = df_master_tfw5.groupby(['listing_id','year','month']).agg(['count'])['filled_in_price_per_day'].reset_index()

In [ ]:
df_master_inquiry_count.head()

In [ ]:
# rename count to inquiry_count

df_master_inquiry_count.rename(columns={'count':'inquiry_count'},inplace=True)

In [ ]:
# merge these counts back into master set 

df_master_tfw6 = pd.merge(df_master_inquiry_count, df_master_tfw5, left_on=['listing_id','year','month'], right_on=['listing_id','year','month'], how='right')

In [ ]:
# read in the appropriate stats data

df_stats_extract = pd.read_csv('../data/statistic_id_month_20210721.csv')


In [ ]:

df_master_tfw6 = pd.merge(df_stats_extract, df_master_tfw6, left_on=['listing_id','year','month'], right_on=['listing_id','year','month'], how='right')

In [ ]:
df_master_tfw6.nunique()

In [ ]:
df_master_tfw6.drop(['yearmonth'],axis=1,inplace=True)

In [ ]:
# calculate inquiry rate from inquiry count and expose views

df_master_tfw6['inquiry_rate'] = round(df_master_tfw6.inquiry_count / df_master_tfw6.expose_views, 2)
df_master_tfw6 = df_master_tfw6.query('inquiry_rate <= 1.00 and inquiry_rate != inf')

In [ ]:
df_master_tfw6.nunique()

In [ ]:
df_master_tfw6.info()

In [ ]:
df_master_tfw6.nunique()

In [ ]:
# save merged set

df_master_tfw6.to_csv('../data/master_6.csv')

# now it is time to merge in the top listing status

In [ ]:
# read in master 6

df_master_tfw6 = pd.read_csv('../data/master_6.csv')

In [ ]:
# integrate top listing information into dataset

df_top_listings = pd.read_csv('../data/TOP_listings_20210712.csv')

In [ ]:
df_top_listings.head()

In [ ]:
# add a flag to mark top listings

df_top_listings['top'] = 1

In [ ]:
# drop date_from and date_to form the master, since they are not needed anymore and will conflict with the top listings dates

df_master_tfw6.drop(['date_from'],axis=1,inplace=True)

In [ ]:
df_master_tfw6.drop(['date_to'],axis=1,inplace=True)

In [ ]:
# put into datetime format

df_top_listings['date_from'] = pd.to_datetime(df_top_listings['date_from'], errors = 'coerce')
df_top_listings['date_to'] = pd.to_datetime(df_top_listings['date_to'], errors = 'coerce')
df_master_tfw6['date'] = pd.to_datetime(df_master_tfw6['date'], errors = 'coerce')

In [ ]:
# merge into one big set so we can filter

df_topping = pd.merge(df_master_tfw6, df_top_listings, left_on='listing_id', right_on='listing_id', how='outer')

In [ ]:
df_topping.shape

In [ ]:
df_topping.to_csv('../data/temp_top.csv')

In [ ]:
df_topping.nunique()

In [ ]:
# filter to get those rows with top listings active

df_top_filter = df_topping[(df_topping.date >= df_topping.date_from) & (df_topping.date <= df_topping.date_to)]


In [ ]:
df_top_filter.nunique()

In [ ]:
df_top_filter['top'].unique()

In [ ]:
df_top_filter_small = df_top_filter[['listing_id','top']]

In [ ]:
# merge filtered stuff back into the master

df_master_tfw7 = pd.merge(df_top_filter_small, df_master_tfw6, left_on=['listing_id'], right_on=['listing_id'], how='right')

In [ ]:
# save to be safe

df_master_tfw7.to_csv('../data/master_7.csv')

In [ ]:
df_master_tfw7.nunique()

In [ ]:
# after inspection, these are columns we can drop

drop_list =['contract_end','zip','binding_inquiry','result_views','expose_views','ski_boot_heaters','inquiry_price','departure_date','region','subregion']

for i in drop_list:
    df_master_tfw7.drop([i],axis=1,inplace=True)

In [ ]:
df_master_tfw7['top'].fillna(0,inplace=True)

In [ ]:
# save as merged master

df_master_tfw7.to_csv('../data/super_master.csv')

In [ ]:
df_master_tfw7.nunique()